Import Library

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from google.colab import files

Import Dataset

In [2]:
uploaded = files.upload()

Saving voice (1).csv to voice (1).csv


In [3]:
filename = list(uploaded.keys())[0]

Load Dataset

In [7]:
df = pd.read_csv('voice (1).csv')

Encoding untuk male dan female

In [8]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [11]:
X = df.drop('label', axis=1)
y = df['label']

Training SVM

In [27]:
def run_svm(test_ratio, kernel):
    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_ratio, random_state=42
    )

    # Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Model SVM
    model = SVC(kernel=kernel)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    # Akurasi
    acc = accuracy_score(y_test, pred)
    return acc

print("Akurasi linear 70:30 :", run_svm(0.30, 'linear'))
print("Akurasi RBF 80:20    :", run_svm(0.20, 'rbf'))


Akurasi linear 70:30 : 0.9705573080967402
Akurasi RBF 80:20    : 0.9826498422712934



Gunakan data pada praktikum 5 untuk membuat model klasifikasi siang dan malam menggunakan SVM dengan kernel RBF menggunakan fitur histrogram. Gunakan rasio 80:20. Anda dapat bereksperimen dengan hyperparameter tunning dari kernel RBF. Catat performansi akurasinya!



Import Library

In [28]:
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Dataset

In [56]:
train_dir = "/content/drive/MyDrive/ML/images/training/"
test_dir  = "/content/drive/MyDrive/ML/images/test/"

Ekstraksi Fitur

In [57]:
def extract_histogram(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Normalisasi ukuran

     # Histogram 256 bins per channel (R,G,B)
    hist_r = cv2.calcHist([img], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([img], [1], None, [256], [0, 256])
    hist_b = cv2.calcHist([img], [2], None, [256], [0, 256])

    # Gabungkan
    hist = np.concatenate([hist_r, hist_g, hist_b]).flatten()

    # Normalisasi
    hist = hist / np.sum(hist)

    return hist

In [61]:
# Load data gambar
def load_dataset(directory):
    features = []
    labels = []

    # Mapping label sesuai nama folder
    label_map = {
        "day": 0,
        "night": 1
    }

    for label in label_map.keys():
        folder_path = os.path.join(directory, label)
        print("Membaca folder:", folder_path)

        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            hist = extract_histogram(img_path)
            features.append(hist)
            labels.append(label_map[label])

    return np.array(features), np.array(labels)


In [62]:
X_train_full, y_train_full = load_dataset(train_dir)
X_test, y_test = load_dataset(test_dir)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

print("Data siap!")
print("Train :", X_train.shape)
print("Val   :", X_val.shape)
print("Test  :", X_test.shape)


Membaca folder: /content/drive/MyDrive/ML/images/training/day
Membaca folder: /content/drive/MyDrive/ML/images/training/night
Membaca folder: /content/drive/MyDrive/ML/images/test/day
Membaca folder: /content/drive/MyDrive/ML/images/test/night
Data siap!
Train : (192, 768)
Val   : (48, 768)
Test  : (160, 768)


In [63]:
param_grid = {
    "C": [0.1, 1, 10, 100],
    "gamma": ["scale", 0.1, 0.01, 0.001]
}

svm = SVC(kernel="rbf")

grid = GridSearchCV(svm, param_grid, cv=3, n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

print("Best parameters:", grid.best_params_)

best_svm = grid.best_estimator_

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters: {'C': 10, 'gamma': 'scale'}


In [65]:
# Evaluasi Validasi dan Testing
y_pred_val = best_svm.predict(X_val)
val_acc = accuracy_score(y_val, y_pred_val)
print("\n Akurasi Validasi:", val_acc)


 Akurasi Validasi: 1.0
